# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

In [2]:
# Required library
import openai
import os
import pandas as pd
import requests
from typing import List, Union, Dict
from scipy.spatial.distance import cosine

# OpenAI API key
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-2069282693126677350909067aba0968b4a35.33439325"

# OpenAI Models
EMBEDDING_MODEL = 'text-embedding-3-small'
COMPLETION_MODEL = 'gpt-3.5-turbo'

# Batch size for processing
BATCH_SIZE = 25

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [3]:
dataset = pd.read_csv("./data/finanical_rag_embedding_dataset.csv")
dataset.head()

,question,context
0,What area did NVIDIA initially focus on before...,"Since our original focus on PC graphics, we ha..."
1,What are some of the recent applications of GP...,Some of the most recent applications of GPU-po...
2,What significant invention did NVIDIA create i...,Our invention of the GPU in 1999 defined moder...
3,How does NVIDIA's platform strategy contribute...,"NVIDIA has a platform strategy, bringing toget..."
4,What does NVIDIA's CUDA programming model enable?,With our introduction of the CUDA programming ...


In [4]:
dataset['text'] = dataset['context']
dataset = dataset.drop(columns=['question','context'])
dataset

,text
0,"Since our original focus on PC graphics, we ha..."
1,Some of the most recent applications of GPU-po...
2,Our invention of the GPU in 1999 defined moder...
3,"NVIDIA has a platform strategy, bringing toget..."
4,With our introduction of the CUDA programming ...
...,...
6995,The 5.400% Senior Notes due in 2028 have an in...
6996,"On January 30, 2023, LVSC entered into amendme..."
6997,"Following the downgrades, each series of the o..."
6998,The amended and restated facility agreement wi...


In [7]:
dataset = dataset = dataset.iloc[:2000]
dataset

,text
0,"Since our original focus on PC graphics, we ha..."
1,Some of the most recent applications of GPU-po...
2,Our invention of the GPU in 1999 defined moder...
3,"NVIDIA has a platform strategy, bringing toget..."
4,With our introduction of the CUDA programming ...
...,...
1995,Visa's rule (and a similar Mastercard rule) th...
1996,The consolidated balance sheets of Visa Inc. a...
1997,Net income for Visa Inc. as of the end of Sept...
1998,"As of September 30, 2023, Visa's total equity ..."


In [8]:
# Initialize OpenAI client
openai_client = openai.api_key

batch_size = 100
embeddings = []
for i in range(0, len(dataset), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=dataset.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
dataset["embeddings"] = embeddings
dataset

/tmp/ipykernel_13/780461883.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["embeddings"] = embeddings


,text,embeddings
0,"Since our original focus on PC graphics, we ha...","[0.030941098928451538, -0.006647604517638683, ..."
1,Some of the most recent applications of GPU-po...,"[-0.008451761677861214, -0.03164077177643776, ..."
2,Our invention of the GPU in 1999 defined moder...,"[-0.014972795732319355, -0.03127200901508331, ..."
3,"NVIDIA has a platform strategy, bringing toget...","[0.019686702638864517, -0.031540438532829285, ..."
4,With our introduction of the CUDA programming ...,"[0.0019364568870514631, -0.018282275646924973,..."
...,...,...
1995,Visa's rule (and a similar Mastercard rule) th...,"[-0.022455504164099693, -0.01570861041545868, ..."
1996,The consolidated balance sheets of Visa Inc. a...,"[0.009025214239954948, 0.0060989102348685265, ..."
1997,Net income for Visa Inc. as of the end of Sept...,"[0.02015370875597, -0.012018897570669651, 0.03..."
1998,"As of September 30, 2023, Visa's total equity ...","[0.011561392806470394, -0.00808965228497982, 0..."


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [18]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question: str, df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a copy of the input DataFrame sorted by the cosine distance between the 
    question embedding and each row's embedding (lower distance means more relevant).

    Args:
        question (str): The question to compute relevance for.
        df (pd.DataFrame): DataFrame containing a column "embeddings" with precomputed embeddings.

    Returns:
        pd.DataFrame: A new DataFrame sorted by relevance (ascending cosine distance).
    """
    # Get the embedding for the question
    question_embedding = get_embedding(question, engine=EMBEDDING_MODEL)
    
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()
    
    # Compute cosine distances between the question embedding and each row's embedding.
    df_copy["distances"] = distances_from_embeddings(
        question_embedding,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the DataFrame by ascending distance (more relevant rows first)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    
    return df_copy


In [19]:
dataset['embeddings'][0]

[0.030941098928451538,
 -0.006647604517638683,
 0.04810722917318344,
 0.01636434532701969,
 -0.017008403316140175,
 -0.006381437182426453,
 0.01120530441403389,
 0.04603046923875809,
 -0.040220800787210464,
 -0.021582534536719322,
 0.01879599504172802,
 -0.011974231339991093,
 -0.0381966158747673,
 -0.023540997877717018,
 -0.000361256068572402,
 0.026800721883773804,
 0.018493682146072388,
 -0.027339627966284752,
 0.01837538555264473,
 0.011054147966206074,
 -0.011928227730095387,
 -0.04024708643555641,
 0.04248157516121864,
 -0.03817032650113106,
 -0.00046291708713397384,
 -0.012053095735609531,
 -0.014143000356853008,
 0.0009340491960756481,
 0.04287589713931084,
 -0.02089904248714447,
 0.01741586998105049,
 -0.031940046697854996,
 -0.04931648448109627,
 0.0031578585039824247,
 -0.05441637709736824,
 0.016075175255537033,
 -0.027050459757447243,
 0.04873814433813095,
 -0.028207136318087578,
 0.025249723345041275,
 0.028180846944451332,
 0.02653784118592739,
 -0.013709246180951595,
 -

In [31]:
# question 1
Que_1 = "What are some of the recent applications of GPU-powered deep learning as mentioned by NVIDIA?"
sorted_dataset1 = get_rows_sorted_by_relevance(Que_1, dataset)
sorted_dataset1.head()

,text,embeddings,distances
1,Some of the most recent applications of GPU-po...,"[-0.008451761677861214, -0.03164077177643776, ...",0.250880
54,Fueled by the sustained demand for exceptional...,"[-0.009249124675989151, -0.036592036485672, 0....",0.396349
21,"We provide a complete, end-to-end accelerated ...","[-0.00027556781424209476, -0.03074817545711994...",0.416980
5,A rapidly growing number of enterprises and st...,"[-0.016046592965722084, -0.03647651895880699, ...",0.423807
9,Data Center The NVIDIA computing platform is f...,"[-0.029146181419491768, -0.02139190025627613, ...",0.431113


In [32]:
# question 1
Que_2 = "What does NVIDIA's CUDA programming model enable?"
sorted_dataset1 = get_rows_sorted_by_relevance(Que_2, dataset)
sorted_dataset1.head()

,text,embeddings,distances
4,With our introduction of the CUDA programming ...,"[0.0019364568870514631, -0.018282275646924973,...",0.330135
8,Data Center The NVIDIA computing platform is f...,"[-0.029146181419491768, -0.02139190025627613, ...",0.419232
9,Data Center The NVIDIA computing platform is f...,"[-0.029146181419491768, -0.02139190025627613, ...",0.419232
21,"We provide a complete, end-to-end accelerated ...","[-0.00027556781424209476, -0.03074817545711994...",0.449184
54,Fueled by the sustained demand for exceptional...,"[-0.009249124675989151, -0.036592036485672, 0....",0.472756


In [22]:
import tiktoken

prompt_template = """
    Answer the question based on the context below, and if the question
    can't be answered based on the context, say "I don't know"

    Context: 

    {}

    ---

    Question: {}
    Answer:
"""


def create_prompt(question, df, max_token_count, prompt_template=prompt_template):
    """
    Creates a prompt by incorporating relevant context from the DataFrame, ensuring that the 
    total token count does not exceed max_token_count. The context is gathered from rows 
    sorted by relevance to the question.

    Args:
        question (str): The question to generate a prompt for.
        df (pd.DataFrame): DataFrame containing at least a "text" column and an "embeddings" column.
        max_token_count (int): Maximum allowed token count for the generated prompt.
        prompt_template (str): A template string with two placeholders:
                               one for the context and one for the question.
    
    Returns:
        str: The formatted prompt with context inserted.
    """
    # Get the tokenizer for counting tokens (using tiktoken's "cl100k_base")
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Initialize the current token count with the tokens in the prompt template and question.
    # Note: This assumes prompt_template is a static string that will be formatted with context and question.
    current_token_count = len(tokenizer.encode(prompt_template)) + len(tokenizer.encode(question))
    
    # Prepare an empty list for context entries
    context = []
    
    # Loop through each text entry sorted by relevance (assuming get_rows_sorted_by_relevance returns a DataFrame)
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        # Count tokens in the current text entry
        text_token_count = len(tokenizer.encode(text))
        
        # Check if adding this text stays within the max_token_count
        if current_token_count + text_token_count <= max_token_count:
            context.append(text)
            current_token_count += text_token_count
        else:
            # Stop adding context if the limit is exceeded
            break

    # Format the prompt by joining context entries with a delimiter (e.g., "\n\n###\n\n")
    formatted_context = "\n\n###\n\n".join(context)
    return prompt_template.format(formatted_context, question)


In [24]:
max_token_count = 300
print(create_prompt(question1, dataset, max_token_count))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

With our introduction of the CUDA programming model in 2006, we opened the parallel processing capabilities of our GPU for general purpose computing.

###

Data Center The NVIDIA computing platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific computing, across hyperscale, cloud, enterprise, public sector, and edge data centers. The platform consists of our energy efficient GPUs, data processing units, or DPUs, interconnects and systems, our CUDA programming model, and a growing body of software libraries, software development kits, or SDKs, application frameworks and services, which are either available as part of the platform or packaged and sold separately.

###

Data Center The NVIDIA computing platform is focused on accelerating the most compute-intensive workloads, such as

In [44]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(question, df, max_prompt_tokens=1000, max_answer_tokens=150):
    """
    Answers a question by creating a prompt with relevant context from the DataFrame,
    sending it to the OpenAI API, and returning the generated answer.

    Args:
        question (str): The question to be answered.
        df (pd.DataFrame): DataFrame containing a "text" column and precomputed "embeddings".
        max_prompt_tokens (int): Maximum allowed tokens for the prompt (including context).
        max_answer_tokens (int): Maximum number of tokens to generate for the answer.

    Returns:
        str: The generated answer from the OpenAI API.
    """
    # Create a prompt that includes relevant context from the dataset
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        # Send the prompt to the OpenAI API using the specified completion model
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        # Extract and return the answer text from the first choice in the response
        return response["choices"][0]["text"].strip()
    except Exception as e:
        # Print the error and return an empty string if something goes wrong
        print(e)
        return ""


In [45]:
answer1 = answer_question(Que_1, dataset)
print(answer1)

Some recent applications of GPU-powered deep learning as mentioned by NVIDIA include recommendation systems, large language models, and generative AI.


In [46]:
answer2 = answer_question(Que_2, dataset)
print(answer2)

General purpose computing and parallel processing on their GPUs.


## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [58]:
que_1 = "What are the home products mentioned in the text?"

answer1 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=que_1,
    max_tokens=400
)["choices"][0]["text"].strip()
print(answer1)

The text does not mention any specific home products. Without further context, it is not possible to accurately determine the specific home products mentioned in the text.


In [59]:
custom_chatbot_anwer1 = answer_question(que_1, dataset)
print(custom_chatbot_anwer1)

Apple TV, HomePod, HomePod mini, Clayton Homes, Pantene, Head & Shoulders.


### Question 2

In [61]:
que_2 = "Why are the outcomes of various legal proceedings and claims inherently uncertain?"

answer2 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=que_2,
    max_tokens=400
)["choices"][0]["text"].strip()

print(answer2)

1. Interpretation of Laws: The interpretation of laws can vary from case to case, and the outcome of a legal proceeding may depend on how the law is interpreted in a specific case. Different judges and juries may have their own interpretation of the law, which can lead to varying outcomes.

2. Facts and Evidence: The outcome of a legal proceeding often relies on the presentation and evaluation of facts and evidence. However, there can be differing accounts of what happened, conflicting evidence, and unreliable witnesses, making it difficult to determine the truth and leading to uncertainty in the outcome.

3. Complexity of Cases: Legal proceedings can involve complex issues and arguments that require a deep understanding of the law and legal procedures. Each case is unique and may have its own set of complexities, making it difficult to predict the outcome.

4. Human Bias: Human bias can affect the outcome of legal proceedings. Judges and juries are human and may have their own beliefs

In [62]:
custom_chatbot_anwer2 = answer_question(que_2, dataset)
print(custom_chatbot_anwer2)

The outcomes of legal proceedings and claims are inherently uncertain because they are subject to significant judgment and are unpredictable.


### Question 3

In [67]:
que_3 = "How much has been invested towards the goal of advancing equity in well-being as of January 28, 2024?"

answer3 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=que_3,
    max_tokens=400
)["choices"][0]["text"].strip()

print(answer3)

As of January 28, 2024, it is not possible to determine the exact amount that has been invested towards the goal of advancing equity in well-being, as this will depend on various factors such as government budgets, private sector investments, and philanthropic donations. It is also likely that new investments will continue to be made towards this goal in the future.


In [68]:
custom_chatbot_anwer3 = answer_question(que_3, dataset)
print(custom_chatbot_anwer3)

$44.8 million
